In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# 반드시 s3a://로 명확히!
bronze_path = "s3a://bronze/"
jPath = spark._jvm.org.apache.hadoop.fs.Path(bronze_path)

# 여기서 FileSystem 얻기!
fs = jPath.getFileSystem(spark._jsc.hadoopConfiguration())
status_list = fs.listStatus(jPath)
file_list = [f.getPath().toString() for f in status_list if f.isFile()]

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/07 07:17:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/07 07:17:55 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [2]:
catalog = 'dev'
namespace = 'silver'
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {catalog}.{namespace}")

for file_path in file_list:
    df = spark.read.csv(file_path, header=True, inferSchema=True)
    dedup_df = df.dropDuplicates()
    print(f"{file_path}, before: {df.count()}, after: {dedup_df.count()}")

    table_name = file_path.split("/")[-1].replace(".csv", "")
    full_table_name = f"{catalog}.{namespace}.{table_name}"

    if not spark.catalog.tableExists(full_table_name):
        dedup_df.writeTo(full_table_name).create()
    else:
        dedup_df.writeTo(full_table_name).overwritePartitions()

s3a://bronze/olist_customers_dataset.csv, before: 99441, after: 99441


s3a://bronze/olist_geolocation_dataset.csv, before: 1000163, after: 738332
s3a://bronze/olist_order_items_dataset.csv, before: 112650, after: 112650
s3a://bronze/olist_order_payments_dataset.csv, before: 103886, after: 103886
s3a://bronze/olist_order_reviews_dataset.csv, before: 104162, after: 104077
s3a://bronze/olist_orders_dataset.csv, before: 99441, after: 99441
s3a://bronze/olist_products_dataset.csv, before: 32951, after: 32951
s3a://bronze/olist_sellers_dataset.csv, before: 3095, after: 3095
s3a://bronze/product_category_name_translation.csv, before: 71, after: 71


In [3]:
df = spark.sql("SHOW TABLES IN dev.silver")
table_names = [row.tableName for row in df.collect()]

In [4]:
# for table_name in table_names:
#     spark.sql(f"DROP TABLE IF EXISTS dev.silver.{table_name} PURGE")

spark.stop()